In [1]:
%matplotlib inline


# Detect Objects Using Your Webcam


This demo will take you through the steps of running an "out-of-the-box" detection model to
detect objects in the video stream extracted from your camera.



## Create the data directory
The snippet shown below will create the ``data`` directory where all our data will be stored. The
code will create a directory structure as shown bellow:

```bash
data
└── models
```
where the ``models`` folder will will contain the downloaded models.



In [2]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

## Set Path



In [3]:
MODEL_NAME = 'obj-detection-exported-models'
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))

LABEL_FILENAME = 'label_map.pbtxt'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))

print(PATH_TO_CKPT)
print(PATH_TO_CFG)
print(PATH_TO_LABELS)

C:\Users\yxong\Downloads\data\models\obj-detection-exported-models\checkpoint/
C:\Users\yxong\Downloads\data\models\obj-detection-exported-models\pipeline.config
C:\Users\yxong\Downloads\data\models\obj-detection-exported-models\label_map.pbtxt


## Load the model
Next we load the downloaded model



In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

## Load label map data (for plotting)



In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

## Define the video stream
We will use [OpenCV](https://pypi.org/project/opencv-python/) to capture the video stream
generated by our webcam.



In [6]:
import cv2

cap = cv2.VideoCapture(0)

## Putting everything together
The code shown below loads an image, runs it through the detection model and visualizes the
detection results, including the keypoints.

Note that this will take a long time (several minutes) the first time you run this code due to
tf.function's trace-compilation --- on subsequent runs (e.g. on new images), things will be
faster.

Here are some simple things to try out if you are curious:

* Modify some of the input images and see if detection still works. Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
* Print out `detections['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
* Set ``min_score_thresh`` to other values (between 0 and 1) to allow more detections in or to filter out more detections.



In [7]:
import numpy as np
import time

fall_detected = False

while True:
    # Read frame from camera
    ret, image_np = cap.read()

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    #print((detections['detection_classes'][0][0].numpy() + label_id_offset).astype(int))
    #print(detections['detection_scores'][0][0].numpy())
    
    if fall_detected == False and (detections['detection_classes'][0][0].numpy() + label_id_offset).astype(int) == 1 and detections['detection_scores'][0][0].numpy() > 0.8:
        fall_detected = True
    
    if fall_detected:
        # Add visual alert, e.g., flashing red border
        border_thickness = 20
        border_color = (0, 0, 255)  # Red color in BGR format

        if time.time() % 1 > 0.5:
            # Draw the border with the specified thickness and color
            image_np_with_detections = cv2.rectangle(
                image_np_with_detections,
                (0, 0),
                (image_np_with_detections.shape[1] - 1, image_np_with_detections.shape[0] - 1),
                border_color,
                border_thickness
            )
            
            
            fall_detected = False
            
    # Perform the regular visualization
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.80,
        agnostic_mode=False
    )

    # Display output
    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()